<a href="https://colab.research.google.com/github/kathdevx/emotionalAI/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install soundfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files.txt', 'w') as outfile:
    with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/meta-info/Partitions.txt', 'r') as part_file:
        for line in part_file:
            idx = line.find(';')
            if line[:idx] == 'Train':
                outfile.write(line[idx+2:-1])
                outfile.write('\n')

In [3]:
train_fnames = []

with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files.txt', 'r') as infile:
    train_fnames = infile.read().splitlines()

Kapoia arxeia den vrethikan

In [ ]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

print(file_len('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/sentences.txt'))
print(len(train_fnames))

In [ ]:
dir = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/'

import soundfile as sf
import glob

train_data = open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files_path.txt', 'w')

for filename in glob.glob(dir+'*.wav'):
    idx = filename.rfind('/')
    fname = filename[idx+1:]

    if fname in train_fnames:
        train_data.write(fname)
        train_data.write(' ')
        train_data.write(filename)
        train_data.write('\n')

train_data.close()

In [4]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-338uf24v
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-338uf24v
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 42.3 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2587137 sha256=4aa7120a04ee926a4d7ed2f14fc3638df215660d39ea042b141dce8d0d444910
  Stored in directory: /tmp/pip-ephem-wheel-cache-svfqssep/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import librosa
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import nltk

def load_model():
    tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-large-960h')
    model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-large-960h')
    return tokenizer, model

def correct_sentence(input_text):
    sentences = nltk.sent_tokenize(input_text)
    return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

def asr_transcript(input_file):

    tokenizer, model = load_model();
    
    speech, fs = sf.read(input_file)

    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]

    if fs != 16000:
        speech = librosa.resample(speech, fs, 16000)

    input_values = tokenizer(speech, return_tensors="pt").input_values;
    logits = model(input_values).logits;
    
    predicted_ids = torch.argmax(logits, dim=-1)
    
    transcription = tokenizer.decode(predicted_ids[0]);

    return correct_sentence(transcription.lower())

In [7]:
def chunks(lst, n):
    chunky = []
    for i in range(0, len(lst), n):
        chunky.append(lst[i:i + n])
    return chunky

In [8]:
batch_size = 1000
train_data = []

with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files_path.txt', 'r') as tdata:
    data = tdata.read().splitlines()
    train_data = chunks(data, batch_size)

In [ ]:
with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/sentences.txt', 'a') as f:
    for t in train_data[0]: 
        temp = t.split(' ', 1)
        fname = temp[0] + ' '
        f.write(fname)
        f.write(asr_transcript(temp[1]))
        f.write('\n')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:419: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected to